# MD simulation of a drug molecule (chlorzoxazone)

Chlorzoxazone is a small drug molecule. In this notebook we'll run a short simulation of a single chlorzoxazone molecule in *implicit* solvent, a "mean field" way of representing the molecule's interactions with water. (You can read more about implicit solvent models here: https://en.wikipedia.org/wiki/Implicit_solvation.) We will then visualize the simulation in this same notebook. 

In [ ]:
from openmm import app
import openmm as mm
from simtk import unit
from sys import stdout
import time as time

In [ ]:
prmtop = app.AmberPrmtopFile('chlorzoxazone.prmtop')
inpcrd = app.AmberInpcrdFile('chlorzoxazone.inpcrd')

system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=app.HBonds, implicitSolvent=app.HCT)
temperature = 298.15*unit.kelvin
integrator = mm.LangevinIntegrator(temperature, 1/unit.picosecond, 2*unit.femtoseconds)
platform = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

### Questions for you to answer!

1. What is happening in the next three blocks of code?
2. How long is the production simulation in *nano*seconds?

In [ ]:
st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is", st.getPotentialEnergy())

print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is", st.getPotentialEnergy())

In [ ]:
simulation.context.setVelocitiesToTemperature(298.15*unit.kelvin)
print('Equilibrating...')
tinit=time.time()
simulation.step(10000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

In [ ]:
simulation.reporters.append(app.DCDReporter('chlorzoxazone_sim.dcd', 250))
simulation.reporters.append(app.StateDataReporter(stdout, 50000, step=True, time=True,
    potentialEnergy=True, temperature=True, speed=True, separator='\t'))


print('Running Production...')
simulation.step(2500000)
print('Done!')

Now we can visualize the trajectory just as we did before. Remember that we need to load the MDTraj and NGLview libraries first!

In [ ]:
import mdtraj as md
import nglview as nv

traj = md.load('chlorzoxazone_sim.dcd', top='chlorzoxazone.prmtop')
atoms, bonds = traj.topology.to_dataframe()
atoms.head()

In [ ]:
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_ball_and_stick('CZZ')
view

# Your turn #1: is this molecule (very) flexible?

What do you see happening in this trajectory? 

Assuming we ignored the 3D rotations of the molecule, does it appear to be very flexible? For example, do there appear to be big changes in the distances between atoms at opposite ends of the molecule?

What aspect(s) of this molecule's structure do you think might explain your observations about its flexibility?

# Your turn #2: repeat the MD simulation and visualization workflow with another molecular system

Carry out the following procedure:

1. Make a copy of this notebook.
2. Replace every instance of `chlorzoxazone` with `valproate` in both notebooks. (Don't forget the notebook title/filename!)
3. Execute the notebook to completion. Note that there is an additional change you'll need to make in the visualization section; see if you can figure it out on your own!
4. Try to answer **Your turn #1** again for valproate.

# Your turn #3: what do these drug molecules do?

Do Google searches for chlorzoxazone and valproate. What are these drug molecules used for?